In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName('MAST30034 Tutorial 3')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/12 19:02:16 WARN Utils: Your hostname, DESKTOP-EJ27LBR resolves to a loopback address: 127.0.1.1; using 172.26.7.126 instead (on interface eth0)
24/09/12 19:02:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/12 19:02:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import geopandas as gpd
import folium

In [ ]:
# Collected shapefile from
# https://datashare.maps.vic.gov.au/order/confirmation/WnBaWlpBb3ErbkpjNW94WElTcDRIUT09


In [7]:
shapefile = gpd.read_file('../../data/landing/GDA2020/vic_localities.shp')

In [8]:
shapefile

,LC_PLY_PID,LOC_PID,DT_CREATE,LOC_NAME,LOC_CLASS,STATE,geometry
0,lcp-ejVyte1DxXx,locb0dcb52a6b55,2023-06-26,Abbeyard,Gazetted Locality,VIC,"POLYGON ((146.65061 -36.94496, 146.68449 -36.9..."
1,lcp386f2bcf9bce,locb9872f35df41,2021-06-24,Abbotsford,Gazetted Locality,VIC,"POLYGON ((144.99137 -37.80959, 144.99254 -37.8..."
2,lcprFRP2-cPMfr5,loc8123ed12ea8d,2023-03-21,Aberfeldie,Gazetted Locality,VIC,"POLYGON ((144.88854 -37.75546, 144.88854 -37.7..."
3,lcpj3KHi-vakgFj,loc1eb4a229104a,2023-12-10,Aberfeldy,Gazetted Locality,VIC,"POLYGON ((146.35535 -37.69944, 146.35751 -37.6..."
4,lcpoO-hq-yZZASg,locb17be87767e1,2023-12-10,Acheron,Gazetted Locality,VIC,"POLYGON ((145.638 -37.28417, 145.64233 -37.279..."
...,...,...,...,...,...,...,...
2968,lcpen8klpgj0ZHd,loc81a970cc91ee,2024-03-19,Yundool,Gazetted Locality,VIC,"POLYGON ((145.77616 -36.25349, 145.79181 -36.2..."
2969,lcp8d12ab6daadf,loc974441b38fa0,2021-06-24,Yuroke,Gazetted Locality,VIC,"POLYGON ((144.84397 -37.5857, 144.84547 -37.57..."
2970,lcp1NsdmW0lG6PP,locffa8d48503ab,2023-12-10,Yuulong,Gazetted Locality,VIC,"POLYGON ((143.25058 -38.72729, 143.25059 -38.7..."
2971,lcpMajb8Sk2dyrp,locadbdb687a089,2024-03-19,Zeerust,Gazetted Locality,VIC,"POLYGON ((145.36826 -36.25123, 145.36831 -36.2..."


In [22]:
# Convert the geometry shape to to latitude and longitude
# Please attribute this if you are using it
shapefile['geometry'] = shapefile['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [19]:
gdf = gpd.GeoDataFrame(shapefile)
#geoJSON = gdf[['LOCALITY', 'geometry']].drop_duplicates('LOCALITY').to_json()
geoJSON = gdf['geometry'].to_json()

In [ ]:
'''m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=10, attr='https://leaflet-extras.github.io/leaflet-providers/preview/')

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    #data=proportions.reset_index(), # data source
    #columns=['LocationID','tip_profitability'], # the columns required
    key_on='geoJSON', # this is from the geoJSON's properties
    fill_color='Oranges', # color scheme
    nan_fill_color='lightgrey',
    legend_name='Tip Profitability'
)

c.add_to(m)
m'''

In [86]:
import json
import pandas as pd

postcodes = json.load(open('../../data/landing/postcode_mapping.json'))

postcodes = pd.DataFrame(postcodes.items(), columns=['postcode', 'suburb'])

In [87]:
relevant = shapefile[['LOC_NAME', 'geometry']]

# need to change tuples into individual rows of strings
postcodes['suburb'] = postcodes['suburb'].apply(lambda x: list(x))
postcodes_edit = postcodes.explode('suburb')
postcodes_edit['suburb'] = postcodes_edit['suburb'].apply(lambda x: str(x))

postcodes_edit
# merge the two dataframes

#merged = pd.merge(relevant, postcodes, left_on='LOC_NAME', right_on='suburb')


,postcode,suburb
0,3000,Melbourne
1,3001,Melbourne
2,3002,East Melbourne
3,3003,West Melbourne
4,3004,Melbourne
...,...,...
756,8627,Camberwell
757,8659,Administrative Postcode Only
758,8785,Dandenong
759,8865,South Melbourne


In [88]:
postcodes = json.load(open('../../data/landing/postcode_mapping.json'))

# swap the keys and values
postcodes_new = postcodes_edit.set_index('suburb').to_dict()['postcode']

for row in relevant['LOC_NAME']:
    if row in postcodes_new:
        relevant['postcode'] = postcodes_new[row]
    else:
        relevant['postcode'] = None


/home/cansenyurt/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [91]:
# Save the data as a dataframe
#relevant.to_csv('../../data/curated/shapefile_postcodes.csv')
relevant.to_parquet('../../data/curated/shapefile_postcodes.parquet')